## Libraries

In [1]:
import pandas as pd
import numpy as np

## Load features

In [2]:
train = pd.read_pickle('../features/train.pkl')

In [4]:
test = pd.read_pickle('../features/test.pkl')

## Feature selection

In [3]:
from sklearn.feature_selection import VarianceThreshold

try:
    from sklearn.model_selection import train_test_split
except ImportError:
    from sklearn.cross_validation import train_test_split

In [5]:
features = ['cosin_sim', 'word_share', 'q1_char_num', 'q1_word_num', 'q2_char_num', 'q2_word_num',
            'start_with_same_world', 'rfidf_share', 'char_difference', 'word_difference',
           'seq_simhash_distance', 'shingle_simhash_distance', 'avg_word_len_q1', 'avg_word_len_q2',
           'avg_word_difference', 'unigrams_common_count', 'bigrams_common_count', 'unigrams_common_ratio',
           'bigrams_common_ratio', 'word2vec_q1_mean', 'word2vec_q2_mean']

target = 'is_duplicate'

X, y = train[features], train[target]
X_test = test[features]

## Oversampling

In [ ]:
pos_train = X[y == 1]
neg_train = X[y == 0]

# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

X = pd.concat([pos_train, neg_train])
y = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()

del pos_train, neg_train

## Normalization

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler().fit(X)

X = scaler.transform(X)
X_test = scaler.transform(X_test)

## Model

In [8]:
from sklearn.metrics import log_loss

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense, BatchNormalization
from keras_tqdm import TQDMNotebookCallback
import keras

from keras import backend as K
K.set_image_dim_ordering('tf')

Using TensorFlow backend.


In [11]:
# model = Sequential()
# model.add(Dense(256, input_dim=X.shape[1]))
# model.add(BatchNormalization())
# model.add(Activation('relu'))

# model.add(Dense(256))
# model.add(BatchNormalization())
# model.add(Activation('relu'))

# model.add(Dense(512))
# model.add(BatchNormalization())
# model.add(Activation('relu'))

# model.add(Dense(512))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Activation('relu'))

# model.add(Dense(1, activation='sigmoid'))

model = Sequential()
model.add(Dense(512, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
keras_logger = keras.callbacks.TensorBoard(log_dir='../notebooks/tensor_logs/relu_3w_1o_more_neurons',
                                           histogram_freq=1, write_graph=True, write_images=True)

In [13]:
model.fit(X,
          y,
          batch_size=4096 * 8,
          epochs=300,
          verbose=0,
          validation_split=0.1,
          #validation_data = (X_vald_scaled, y_vald),
          callbacks=[TQDMNotebookCallback(), keras_logger])

INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.


INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.


INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.


INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.


INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.


...

INFO:tensorflow:Summary name dense_4/bias:0 is illegal; using dense_4/bias_0 instead.


INFO:tensorflow:Summary name dense_4/bias:0 is illegal; using dense_4/bias_0 instead.


INFO:tensorflow:Summary name dense_4/bias:0 is illegal; using dense_4/bias_0 instead.


INFO:tensorflow:Summary name dense_4/bias:0 is illegal; using dense_4/bias_0 instead.


InvalidArgumentError: Tensor must be 4-D with last dim 1, 3, or 4, not [1,1]
	 [[Node: dense_4/bias_0_1 = ImageSummary[T=DT_FLOAT, bad_color=Tensor<type: uint8 shape: [4] values: 255 0 0...>, max_images=3, _device="/job:localhost/replica:0/task:0/cpu:0"](dense_4/bias_0_1/tag, ExpandDims_19/_59)]]
	 [[Node: dense_4/Sigmoid/_104 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_149_dense_4/Sigmoid", _device="/job:localhost/replica:0/task:0/gpu:0"](dense_4/Sigmoid)]]

Caused by op 'dense_4/bias_0_1', defined at:
  File "/home/weenkus/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/weenkus/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/weenkus/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.laun

In [ ]:
#model.save_weights('relu_6w_2o.h5')
model.save('relu_shallow_model.h5')

In [ ]:
model.load_weights('./relu_shallow.h5')

In [ ]:
scores = model.evaluate(X, y, verbose=0, batch_size=4096 * 8)

print("Model validation accuracy: %.2f" % (scores[1]*100))
print("Model validation loss: %.4f" % (scores[0]))

## Generate submission

In [ ]:
preds = model.predict(X_test, batch_size=4096 * 16)
print(preds.shape)

In [ ]:
np.savetxt(
    '../submissions/submission.csv', np.c_[range(len(preds)), preds],
    delimiter=',', header='test_id,is_duplicate', comments='', fmt='%d,%f'
)